In [23]:
import random
import pandas as pd
import duckdb
from faker import Faker
import numpy as np
from datetime import datetime, timedelta

# Initialize Faker for generating random data
fake = Faker()

# Helper function to generate random date within the last 5 years
def random_date_within_last_years(years=5):
    end_date = datetime.now()
    start_date = end_date - timedelta(days=365 * years)
    return fake.date_between(start_date=start_date, end_date=end_date)

# 1. Generate Customer Demographics Table
def generate_customer_demographics(num_customers):
    customers = []
    for _ in range(num_customers):
        customers.append({
            'customer_id': fake.uuid4(),
            'customer_name': fake.name(),
            'customer_age': random.randint(18, 70),
            'customer_gender': random.choice(['Male', 'Female']),
            'customer_city': fake.city(),
            'customer_state': fake.state(),
            'customer_email': fake.email(),
            'customer_phone': fake.phone_number(),
        })
    return pd.DataFrame(customers)

# 2. Generate Product Master Table
def generate_product_master(num_products):
    products = []
    for _ in range(num_products):
        products.append({
            'product_id': fake.uuid4(),
            'product_name': random.choice(['Motor', 'Health', 'Life', 'Property']),
            'product_price': random.randint(5000, 50000),
            'product_type': random.choice(['Basic', 'Premium']),
        })
    return pd.DataFrame(products)

# 3. Generate Customer Policy Table
def generate_customer_policy(num_policies, product_master):
    policies = []
    for _ in range(num_policies):
        product = product_master.sample(n=1).iloc[0]
        policy_start_date = random_date_within_last_years()
        policy_length = random.choice([6, 12, 24, 36, 48, 60])  # Policy length in days (6 months to 5 years)
        policy_end_date = policy_start_date + pd.DateOffset(months=policy_length)
        
        policies.append({
            'policy_id': fake.uuid4(),
            'customer_id': fake.uuid4(),
            'product_id': product['product_id'],
            'policy_start_date': policy_start_date,
            'policy_end_date': policy_end_date,
            'policy_type': product['product_name'],
            'premium_amount': product['product_price'],
        })
    return pd.DataFrame(policies)

# 4. Generate Premium Register
def generate_premium_register(customer_policy):
    premiums = []
    for index, policy in customer_policy.iterrows():
        premiums.append({
            'policy_id': policy['policy_id'],
            'customer_id': policy['customer_id'],
            'premium_paid_date': random_date_within_last_years(),
            'premium_amount': policy['premium_amount'],
        })
    return pd.DataFrame(premiums)

# 5. Generate Claim Register
def generate_claim_register(customer_policy, num_claims):
    claims = []
    for _ in range(num_claims):
        policy = customer_policy.sample(n=1).iloc[0]
        claim_date = random_date_within_last_years()
        if claim_date <= policy['policy_end_date']:
            claims.append({
                'claim_id': fake.uuid4(),
                'policy_id': policy['policy_id'],
                'customer_id': policy['customer_id'],
                'claim_amount': policy['premium_amount'],
                'claim_date': claim_date,
                'claim_type': random.choice(['Accident', 'Fire', 'Health', 'Natural Disaster']),
                'claim_status': random.choice(['Pending', 'Settled', 'Rejected']),
            })
    return pd.DataFrame(claims)

# 6. Generate Customer Policy Cancellation Table
def generate_cancellation_table(customer_policy):
    cancellations = []
    for policy in customer_policy:
        if random.random() < 0.1:  # Assume 10% of policies get cancelled
            cancellations.append({
                'policy_id': policy['policy_id'],
                'customer_id': policy['customer_id'],
                'cancellation_date': random.choice([policy['policy_end_date'] - timedelta(days=random.randint(1, 180)), policy['policy_end_date']]),
                'cancellation_reason': random.choice(['Customer Request', 'Non-payment', 'Other']),
            })
    return pd.DataFrame(cancellations)

# 7. Generate Marketing Table
def generate_marketing_table(num_customers):
    marketing = []
    for _ in range(num_customers):
        marketing.append({
            'customer_id': fake.uuid4(),
            'marketing_campaign': random.choice(['TV', 'Social Media', 'Billboards', 'Email']),
            'cost_per_customer': random.randint(100, 1000),
            'campaign_channel': random.choice(['Online', 'Offline']),
        })
    return pd.DataFrame(marketing)

# 8. Generate Customer Complaints Table
def generate_complaints_table(num_complaints):
    complaints = []
    for _ in range(num_complaints):
        complaints.append({
            'complaint_id': fake.uuid4(),
            'customer_id': fake.uuid4(),
            'complaint_date': random_date_within_last_years(),
            'complaint_type': random.choice(['Service', 'Claim Settlement', 'Premium Payment', 'Other']),
            'complaint_status': random.choice(['Resolved', 'Pending', 'Escalated']),
        })
    return pd.DataFrame(complaints)

# 9. Generate Fraudulent Claims Table
def generate_fraudulent_claims_table(claim_register):
    fraud_claims = []
    for claim in claim_register:
        if random.random() < 0.05:  # Assume 5% of claims are fraudulent
            fraud_claims.append({
                'claim_id': claim['claim_id'],
                'fraudulent_claim': True,
                'fraud_reason': random.choice(['Exaggerated Claim', 'Fake Claim', 'Duplicate Claim']),
            })
    return pd.DataFrame(fraud_claims)

# 10. Calculate Key Metrics
def calculate_customer_lifetime_value(customer_acquisition, premium_register, claim_register):
    # Customer Acquisition Cost (CAC)
    customer_acquisition_cost = customer_acquisition.groupby('customer_id')['cost_per_customer'].sum()

    # Total Claims
    claims_amount = claim_register.groupby('customer_id')['claim_amount'].sum()

    # Premiums Collected
    premiums_collected = premium_register.groupby('customer_id')['premium_amount'].sum()

    # Calculate CLV: (Premiums Collected + Claims) - Acquisition Cost
    lifetime_value = premiums_collected + claims_amount - customer_acquisition_cost
    return lifetime_value

def calculate_churn_rate(cancellation_table):
    churn_customers = cancellation_table['customer_id'].unique()
    churn_rate = len(churn_customers) / len(customer_demographics)
    return churn_rate

# Parameters
num_customers = 1000
num_products = 5
num_policies = 2000
num_claims = 500
num_acquisitions = 1000
num_complaints = 200
num_fraudulent_claims = 50

# Generate Data
customer_demographics = generate_customer_demographics(num_customers)
product_master = generate_product_master(num_products)


In [24]:
customer_policy = generate_customer_policy(num_policies, product_master)

In [25]:
premium_register = generate_premium_register(customer_policy)
claim_register = generate_claim_register(customer_policy, num_claims)
cancellation_table = generate_cancellation_table(customer_policy)
marketing_table_data = generate_marketing_table(num_customers)
customer_complaints = generate_complaints_table(num_complaints)
fraudulent_claims = generate_fraudulent_claims_table(claim_register)

TypeError: Cannot compare Timestamp with datetime.date. Use ts == pd.Timestamp(date) or ts.date() == date instead.

In [ ]:

# Save to Parquet using DuckDB
duck_db_connection = duckdb.connect()

# Register tables with DuckDB
duck_db_connection.register('customer_demographics', customer_demographics)
duck_db_connection.register('product_master', product_master)
duck_db_connection.register('customer_policy', customer_policy)
duck_db_connection.register('premium_register', premium_register)
duck_db_connection.register('claim_register', claim_register)
duck_db_connection.register('cancellation_table', cancellation_table)
duck_db_connection.register('marketing_table', marketing_table_data)
duck_db_connection.register('customer_complaints', customer_complaints)
duck_db_connection.register('fraudulent_claims', fraudulent_claims)

# Save tables to Parquet
customer_demographics.to_parquet("customer_demographics.parquet")
product_master.to_parquet("product_master.parquet")
customer_policy.to_parquet("customer_policy.parquet")
premium_register.to_parquet("premium_register.parquet")
claim_register.to_parquet("claim_register.parquet")
cancellation_table.to_parquet("cancellation_table.parquet")
marketing_table_data.to_parquet("marketing_table.parquet")
customer_complaints.to_parquet("customer_complaints.parquet")
fraudulent_claims.to_parquet("fraudulent_claims.parquet")

# Calculate Metrics
lifetime_value = calculate_customer_lifetime_value(marketing_table_data, premium_register, claim_register)
churn_rate = calculate_churn_rate(cancellation_table)

# Display Results
print("Customer Lifetime Value (first 10 customers):")
print(lifetime_value.head(10))
print("\nCustomer Churn Rate:", churn_rate)